In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
merged_data_folder = './../dat/merged/'
sins_data_folder = './../dat/sins/'

In [3]:
for sequence_file in tqdm(os.listdir(merged_data_folder)):
    # Read sequence data frame
    sequence_df = pd.read_csv(merged_data_folder + sequence_file, sep=' ')
    
    # Initialize velocity and positions
    dt = sequence_df['time(us)'].diff() / 1e6
    v = np.zeros(3)
    p = np.zeros((len(sequence_df), 3))
    p[0] = sequence_df.loc[0, ['px', 'py', 'pz']].values
    
    # Calculate sins_px, sins_py, and sins_pz
    for i in range(1, len(sequence_df)):
        a = sequence_df.loc[i, ['ax(m/s^2)', 'ay(m/s^2)', 'az(m/s^2)']].values
        v = v + a * dt[i]
        p[i] = p[i - 1] + v * dt[i] + 0.5 * a * (dt[i] ** 2)
    
    # Add sins columns to the data frame
    sequence_df['sins_px'] = p[:, 0]
    sequence_df['sins_py'] = p[:, 1]
    sequence_df['sins_pz'] = p[:, 2]
    
    # Save sins data frames
    sequence_name = sequence_file.split('_')[0]
    sequence_df.to_csv(sins_data_folder + sequence_file + '_sins.txt', sep=' ', index=False, float_format='%.18e')

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:16<00:00,  2.13it/s]
